<a href="https://colab.research.google.com/github/ksk66117-cell/Syntecxhub_project1_personal-voice-assistants/blob/main/voiceass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Personal Voice Assistant
# Requirements: speech-to-text + command mapping + text-to-speech
# Recommended installation:
!apt-get install -y portaudio19-dev espeak
%pip install speechrecognition pyttsx3 pyaudio requests datetime

import speech_recognition as sr
import pyttsx3
import datetime
import webbrowser
import os
import sys
import requests
from time import sleep

# ───────────────────────────
#  Initialize text-to-speech engine
# ───────────────────────────

try:
    engine = pyttsx3.init()
    engine.setProperty('rate', 165)      # speed
    engine.setProperty('volume', 0.9)    # volume 0.0 to 1.0

    # Try to use better voice if available (Windows often has better voices)
    voci_id = None
    voices = engine.getProperty('voices')
    for voice in voices:
        if 'zira' in voice.id.lower() or 'david' in voice.id.lower() or 'female' in voice.name.lower():
            voci_id = voice.id
            break

    if voci_id:
        try:
            engine.setProperty('voice', voci_id)
        except Exception as e:
            print(f"Warning: Could not set preferred voice {voci_id}: {e}")
            print("Using default voice.")
except Exception as e:
    print(f"Error initializing pyttsx3: {e}. Text-to-speech will be disabled.")
    class DummyEngine:
        def say(self, text):
            print(f"TTS (disabled): {text}")
        def runAndWait(self):
            pass
        def setProperty(self, prop, value):
            pass
        def getProperty(self, prop):
            return [] # Return empty list for voices if requested
    engine = DummyEngine()


def speak(text):
    """Convert text to speech"""
    try:
        engine.say(text)
        engine.runAndWait()
    except Exception as e:
        print("TTS error:", e)


def recognize_speech():
    """Listen to microphone and convert speech to text"""
    recognizer = sr.Recognizer()

    with sr.Microphone() as source:
        print("Listening...", end="\r")
        recognizer.adjust_for_ambient_noise(source, duration=0.8)

        try:
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=8)
            print("Recognizing...   ", end="\r")

            # You can also try: google_cloud_speech, whisper, etc.
            text = recognizer.recognize_google(audio, language='en-IN')
            print("You said →", text)
            return text.lower()

        except sr.WaitTimeoutError:
            print("Timeout. Say something!")
            return None
        except sr.UnknownValueError:
            print("Could not understand audio")
            return None
        except sr.RequestError as e:
            print("Speech service error:", e)
            return None
        except Exception as e:
            print("Microphone/Recognizer error:", e)
            return None


def tell_time():
    now = datetime.datetime.now()
    time_str = now.strftime("%I:%M %p")
    speak(f"The time is {time_str}")


def tell_date():
    now = datetime.datetime.now()
    date_str = now.strftime("%A, %d %B %Y")
    speak(f"Today is {date_str}")


def search_google(query):
    url = f"https://www.google.com/search?q={query.replace(' ', '+')}"
    webbrowser.open(url)
    speak(f"Searching Google for {query}")


def open_website(site):
    sites = {
        "youtube":  "https://www.youtube.com",
        "google":   "https://www.google.com",
        "facebook": "https://www.facebook.com",
        "instagram":"https://www.instagram.com",
        "whatsapp": "https://web.whatsapp.com"
    }

    for key, url in sites.items():
        if key in site:
            webbrowser.open(url)
            speak(f"Opening {key}")
            return True
    return False


def open_application(app_name):
    paths = {
        "notepad":      "notepad.exe",
        "calculator":   "calc.exe",
        "paint":        "mspaint.exe",
        "cmd":          "cmd.exe",
        "word":         r"C:\Program Files\Microsoft Office\root\Office16\WINWORD.EXE",
        "excel":        r"C:\Program Files\Microsoft Office\root\Office16\EXCEL.EXE",
        "chrome":       r"C:\Program Files\Google\Chrome\Application\chrome.exe",
        "firefox":      r"C:\Program Files\Mozilla Firefox\firefox.exe",
    }

    for key, path in paths.items():
        if key in app_name:
            try:
                os.startfile(path)
                speak(f"Opening {key}")
                return True
            except:
                speak(f"Sorry, I couldn't find {key} on this system.")
                return False
    return False


def execute_command(command):
    """Main command mapping logic"""
    if not command:
        return False

    # ── Time / Date ────────────────────
    if "time" in command:
        tell_time()
        return True

    if "date" in command or "day" in command:
        tell_date()
        return True

    # ── Greetings / Exit ─────────────────
    if any(x in command for x in ["hello", "hi ", "hey"]):
        speak("Hello! How can I help you today?")
        return True

    if any(x in command for x in ["bye", "exit", "quit", "goodbye", "sleep"]):
        speak("Goodbye! Have a nice day.")
        sleep(1.2)
        sys.exit(0)

    # ── Open websites ────────────────────
    if "open" in command and any(site in command for site in ["youtube", "google", "facebook", "instagram", "whatsapp"]):
        site = command.replace("open", "").strip()
        if open_website(site):
            return True

    # ── Open applications ──────────────────
    if "open" in command:
        app = command.replace("open", "").strip()
        if open_application(app):
            return True

    # ── Google search ──────────────────
    if "search" in command or "google" in command:
        query = command.replace("search", "").replace("google", "").replace("for", "").strip()
        if query:
            search_google(query)
            return True

    # ── Fallback ────────────────────
    speak("Sorry, I didn't understand that command.")
    return False


def main():
    speak("Voice assistant started. Say a command...")
    print("Voice assistant started. Say a command...")

    while True:
        command = recognize_speech()

        if command:
            print("-"*50)
            print("Command →", command)
            executed = execute_command(command)
            if not executed:
                print("Command not recognized")
            print("-"*50 + "\n")

        sleep(0.4)  # small delay between commands


if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        speak("Goodbye!")
        print("\nStopped by user.")
    except Exception as e:
        print("Main loop crashed:", type(e).__name__, str(e))
        speak("Sorry, something went wrong. Closing now.")